In [ ]:
using LinearAlgebra
using SparseArrays
using HDF5
using IterativeSolvers
using Plots
include("../scripts/disc_solvers.jl")

In [ ]:
data = h5read("../data/disc/pantagruel.h5","/");

In [ ]:
id = Int64.(data["branch"][:,1:2])
b = 1.0 ./ data["branch"][:,4]
nline = length(b)


l = data["bus"][:,3] / 100.0
nb = length(l)

is_producing = data["gen"][:,2] .> 0
id_gen = Int64.(data["gen"][is_producing, 1])
id_load = setdiff(1:length(l), id_gen)
ng = length(id_gen)
nl = length(id_load)

g = zeros(nb)
g[id_gen] = data["gen"][is_producing,2] / 100.0
p = - l + g
p .-= sum(p) / nb

og = zeros(ng)

edges = Int64.(zeros(nline, 2))
line_start = data["branch"][:, 1]
line_end = data["branch"][:, 2]
# bus reordering: generator buses first 
for i in 1:ng
    edges[line_start .== id_gen[i], 1] .= i
    edges[line_end .== id_gen[i], 2] .= i
end
for i in 1:nl
    edges[line_start .== id_load[i], 1] .= i + ng
    edges[line_end .== id_load[i], 2] .= i + ng
end

id = edges
inc = sparse([id[:,1]; id[:,2]], [1:nline; 1:nline], [-ones(nline);ones(nline)])

# set initial conditions
og = zeros(ng)
thg = vec(data["bus"][id_gen, 9] / 180.0 * pi)
thl = vec(data["bus"][id_load, 9] / 180.0 * pi)
mg = vec(data["gen_inertia"])
dg = vec(data["gen_prim_ctrl"])
dload = vec(data["load_freq_coef"])
dg += dload[id_gen]
dl = dload[id_load]
pg = p[id_gen]
pl = p[id_load];

In [ ]:
# get the stable solution
b2 = - im ./ data["branch"][:, 4]
Ybus = conj(inc * sparse(1:nline, 1:nline, b2) * inc')
q = zeros(nb);
V = ones(nb);
theta = zeros(nb);
include("../scripts/disc_solvers.jl")
V, theta, iter = NRsolver(Ybus, V, theta, -p, q, Array{Int64,1}([]), 9, tol = 1E-3, maxiter = 10);

In [ ]:
include("../scripts/disc_solvers.jl")
dt = 5E-3
Ndt = 2E2
interval = 1
omega_t = zeros(ng, Int64.(floor(Ndt / interval)))
k = 1
dp = zeros(ng)
dp[1] = -9.0
for i in 1:Ndt
    y = radau5(og, thg, thl, mg, dg, dl, pg+dp, pl, inc, b, dt, maxiter = 14, tol = 1E-6)
    if(mod(i, interval) == 0)
        println(i, " / ", Ndt, " (", floor(100*i/Ndt), "%)")
            omega_t[:, k] = y[1 : ng]
            k += 1
        end
    og = y[1:ng]
    thg = y[ng + 1 : 2 * ng]
    thl = y[2 * ng + 1 : end]
end
    

In [ ]:
plot(omega_t[1:10:end,:]')